In [1]:
import neptune
import numpy as np

api_token = "xxx"
project_name = "ACFRmarine/ALDI-DOAD"

project = neptune.init_project(
    project=project_name,
    mode="read-only",
    api_token=api_token,

)

[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/ACFRmarine/ALDI-DOAD/


In [2]:
runs_table_df = project.fetch_runs_table(query='(`sys/group_tags`:stringSet CONTAINS "SAOD5")').to_pandas()
runs_table_df = runs_table_df[runs_table_df['sys/failed'] == False]
df = runs_table_df

In [3]:
[a for a in list(df.columns) if "SEED" in a]

['training/config/SEED']

In [4]:
def is_in_run(column_name, run):
    names = column_name.split('/')
    layer = run.get_structure()
    for n in names:
        if n not in layer:
            return False
        layer = layer[n]
    return True

def get_method(group_tags):
    if "CoSt_str" in group_tags:
        return "Strong Loss"
    elif "CoSt_weak" in group_tags:
        return "Weak Loss"
    elif "CoSt" in group_tags or "CoSt_BOTH" in group_tags:
        return "Our method"
    elif "ORACLE" in group_tags:
        return "Fully Supervised"
    else:
        "Not sure"

def get_dataset(group_tags):
    if "ft_sparse" in group_tags:
        return "Bbox Sparse"
    elif "ORACLE" in group_tags:
        return "Bbox All"
    elif "OG" in group_tags and "FEW" in group_tags:
        return "Point Sparse"
    elif "CROP" in group_tags and "FEW" in group_tags:
        return "Point Sparse - Cropped images"
    else:
        return "Unknown"

def get_max_from_run(run_id, column_name, api_token, project_name):
    run = neptune.init_run(with_id=run_id, mode="read-only",
                           api_token=api_token,
                           project=project_name)
    if is_in_run(column_name, run):
        return run[column_name].fetch_values()['value'].max()
    else:
        return np.nan

def get_max_AP50_from_id(run_id):
    return get_max_from_run(run_id, 'training/metrics/bbox/AP50',
                            api_token=api_token, project_name=project_name)



In [5]:
df['species'] = df['training/config/DATASETS/TEST'].apply(lambda x: 'Redcup' if x == "(\'squidle_redcup_test\',)" else 'Urchin')
df['method'] = df['sys/group_tags'].apply(get_method)
df['dataset'] = df['sys/group_tags'].apply(get_dataset)
df['last_AP50'] = df['training/metrics/bbox/AP50']
df['Seed'] = df ['training/config/SEED']

In [6]:
df['max_AP50'] = df['sys/id'].apply(get_max_AP50_from_id)

[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/ACFRmarine/ALDI-DOAD/e/AL-1336
[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/ACFRmarine/ALDI-DOAD/e/AL-1335
[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/ACFRmarine/ALDI-DOAD/e/AL-1334
[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/ACFRmarine/ALDI-DOAD/e/AL-1333
[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/ACFRmarine/ALDI-DOAD/e/AL-1332
[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/ACFRmarine/ALDI-DOAD/e/AL-1331
[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/ACFRmarine/ALDI-DOAD/e/AL-1330
[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/ACFRmarine/ALDI-DOAD/e/AL-1329
[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/ACFRmarine/ALDI-DOAD/e/

In [10]:
# Replace 'group_col1', 'group_col2', 'group_col3' with your actual column names
df_summary = df[['sys/id', 'species', 'dataset', 'method', 'max_AP50', 'last_AP50', 'Seed']]
df_summary = df_summary.sort_values(by=['species','dataset','method','Seed'])

In [8]:
group_columns = ['species', 'dataset', 'method']
df_grouped = df.groupby(group_columns).agg(
    AP50_max_mean=('max_AP50', 'mean'),
    AP50_max_std=('max_AP50', 'std'),
    AP50_last_mean=('last_AP50', 'mean'),
    AP50_last_std=('last_AP50', 'std'),
    count=('max_AP50', 'size')  # or use 'count' if you want to exclude NaNs
).reset_index()

